# Análisis Exploratorio de Datos (EDA) - OULAD Dataset

Este notebook tiene como objetivo comprender la estructura y características del dataset OULAD antes de proceder con modelados complejos.

## Objetivos
1. **Visualizar y entender la información**: ¿Cómo es el dataset?   
2. **Distribución de Clases**: ¿Está desbalanceado el dataset (Pass vs Fail vs Withdrawn)?
3. **Análisis Temporal**: ¿Cómo evoluciona la interacción de los estudiantes a lo largo de las semanas?
4. **Correlaciones Iniciales**: ¿Existe relación visible entre el número de clics y el resultado final?


# 1. Visualización de la información

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]

DATA_DIR = Path("../data/interim")

df_students = pd.read_csv(DATA_DIR / "students_merged.csv")
df_assessments = pd.read_csv(DATA_DIR / "assessments_merged.csv")
df_interactions = pd.read_csv(DATA_DIR / "interactions_merged.csv")

print("Datos cargados correctamente:")
print(f"Estudiantes: {df_students.shape}")
print(f"Evaluaciones: {df_assessments.shape}")
print(f"Interacciones: {df_interactions.shape}")

**Observaciones:** Podemos ver que tenemos un dataset relativamente grande con el que podemos entrenar adecuadamente nuestros modelos de inteligencia artificial y que puedan generalizar bien.

## 1.1. Significado de los datos

A continuación se describe el contenido de las tablas generadas tras el preprocesamiento inicial del dataset OULAD.

### Tabla 1: Estudiantes (`students_processed.csv`)
Esta es la tabla maestra que contiene la información estática y demográfica de cada estudiante matriculado.

| Campo | Descripción |
| :--- | :--- |
| `code_module` | Identificador del curso (ej. AAA, BBB). |
| `code_presentation` | Semestre en el que se imparte (ej. 2013J, 2014B). |
| `id_student` | ID único anonimizado del estudiante. |
| `gender` | Género del estudiante (M/F). |
| `region` | Región geográfica de residencia en UK. |
| `highest_education` | Nivel educativo previo (ej. A Level, HE Qualification). |
| `imd_band` | Índice de privación múltiple (nivel socioeconómico de la zona, % más bajo = más pobreza). |
| `age_band` | Rango de edad (0-35, 35-55, 55<=). |
| `num_of_prev_attempts` | Número de veces que ha intentado este módulo antes. |
| `studied_credits` | Créditos totales que el estudiante está cursando actualmente. |
| `disability` | Si tiene discapacidad declarada (Y/N). |
| `final_result` | **Variable Objetivo**: Pass, Distinction, Fail, Withdrawn. |
| `date_registration` | Día relativo al inicio del curso en que se matriculó. |
| `date_unregistration` | Día relativo en que se dio de baja (si aplica). |

---

### Tabla 2: Evaluaciones (`assessments_processed.csv`)
Contiene los resultados de todos los exámenes y tareas puntuables.

| Campo | Descripción |
| :--- | :--- |
| `id_assessment` | ID de la prueba específica. |
| `id_student` | ID del estudiante. |
| `date_submitted` | Día relativo al inicio del curso en que el alumno entregó la tarea. |
| `is_banked` | Si la nota ha sido convalidada de una presentación anterior. |
| `score` | Nota obtenida (0-100). |
| `assessment_type` | Tipo de examen: TMA (Tutor Marked), CMA (Computer Marked), Exam. |
| `date` | Fecha límite de entrega oficial (Deadline). |
| `weight` | Peso de esta evaluación en la nota final del curso. |

---

### Tabla 3: Interacciones (`interactions_processed.csv`)
Es el log de actividad diario en el VLE (Virtual Learning Environment). Es la tabla más voluminosa.

| Campo | Descripción |
| :--- | :--- |
| `code_module` | Módulo visitado. |
| `id_student` | ID del estudiante. |
| `id_site` | ID del material específico visitado. |
| `date` | Día relativo al inicio del curso (puede ser negativo si accedió antes de que empezará). |
| `sum_click` | Número de clics realizados en ese material ese día. |
| `activity_type` | Tipo de recurso: `forumng` (foro), `oucontent` (contenido HTML), `resource` (PDF/Web), `quiz`, etc. |
| `week_from / week_to` | Semanas planificadas para ese material (muchos nulos). |



## 1.2 Primera toma de contacto con los datasets.

In [ ]:
# Hacemos un display de las primeras filas para familiarizarnos con las columnas
print("--- DATOS DE ESTUDIANTES (Demográficos y Resultado) ---")
display(df_students.head(3))

print("\n--- DATOS DE EVALUACIONES (Notas) ---")
display(df_assessments.head(3))

print("\n--- DATOS DE INTERACCIONES (Clics en recursos) ---")
display(df_interactions.head(3))

# Verificamos si hay nulos
print("\n--- VALORES NULOS POR DATASET ---")
print("Estudiantes:", df_students.isnull().sum().sum())
print("Evaluaciones:", df_assessments.isnull().sum().sum())
print("Interacciones:", df_interactions.isnull().sum().sum())

### 1.1 Análisis Detallado de Nulos
Dado que hemos detectado la presencia de valores faltantes, vamos a identificar el porcentaje que representan. En el dataset OULAD, muchos nulos tienen un **significado estructural** y no son errores de carga.

In [ ]:
# --- ANÁLISIS Y JUSTIFICACIÓN DE VALORES NULOS ---
print("Recuento de valores nulos por tabla:")
for name, df in [("Estudiantes", df_students), ("Evaluaciones", df_assessments), ("Interacciones", df_interactions)]:
    nulls = df.isnull().sum()
    print(f"\nTabbla {name}:")
    print(nulls[nulls > 0])

### Justificación de Decisiones de Preprocesamiento (Basado en EDA)

A continuación, se presentan las pruebas estadísticas que justifican las reglas de limpieza aplicadas en el script `dataset.py`. El objetivo es maximizar la retención de datos útiles sin introducir ruido o sesgos en el modelo:

1.  **Preservación de Estudiantes sin Registro:** Se analiza si los estudiantes con `date_registration` nulo poseen actividad relevante que justifique su imputación en lugar de su eliminación.
2.  **Tratamiento de Nivel Socioeconómico (`imd_band`):** Se evalúa el impacto de la categoría `'Unknown'` para mantener la representatividad de la muestra.
3.  **Saneamiento de Evaluaciones:** Validación del impacto porcentual de la eliminación de registros sin calificación (`score`).
4.  **Validación de Imputación Temporal:** Análisis de la mediana de registro por cohorte para asegurar que la imputación no viola la estructura temporal del curso.

In [ ]:
# --- 1. JUSTIFICACIÓN: ¿Por qué no borrar los 45 estudiantes sin date_registration? ---
# Vamos a ver si estos estudiantes tienen actividad (interacciones y notas)
ids_sin_registro = df_students[df_students['date_registration'].isnull()]['id_student']
actividad_vle = df_interactions[df_interactions['id_student'].isin(ids_sin_registro)].shape[0]
actividad_notas = df_assessments[df_assessments['id_student'].isin(ids_sin_registro)].shape[0]

print(f"Alumnos sin fecha de registro: {len(ids_sin_registro)}")
print(f"-> Total de clics que perderíamos si los borramos: {actividad_vle}")
print(f"-> Total de notas que perderíamos si los borramos: {actividad_notas}")
# CONCLUSIÓN: Imputamos la mediana para no perder miles de registros de comportamiento.

# --- 2. JUSTIFICACIÓN: ¿Por qué imputar 'Unknown' en imd_band? ---
print("\nDistribución de imd_band (incluyendo nulos):")
print(df_students['imd_band'].value_counts(dropna=False, normalize=True) * 100)
# CONCLUSIÓN: El 3.4% de nulos es significativo. Convertirlo en 'Unknown' evita sesgos

# --- 3. JUSTIFICACIÓN: ¿Por qué borrar los 173 registros sin score? ---
print(f"\nPorcentaje de notas faltantes: {(173 / len(df_assessments)) * 100:.4f}%")

# --- 4. DATA LEAKAGE PREVENTION CHECK ---
print("\nMediana de date_registration por cohorte (justifica imputación agrupada):")
print(df_students.groupby('code_presentation')['date_registration'].median())

### Síntesis de Resultados y Decisiones Finales

Tras el análisis detallado de los valores faltantes, se confirman las siguientes acciones ejecutadas en el pipeline de ingeniería de datos:

*   **Retención de Estudiantes "Huéspedes":** Los estudiantes sin fecha de registro presentan una actividad significativa en el VLE y evaluaciones. Su eliminación habría supuesto una pérdida de datos conductuales valiosos. La imputación mediante la mediana por cohorte preserva estos registros sin distorsionar la distribución temporal.
*   **Tratamiento de Datos Socioeconómicos:** El volumen de nulos en `imd_band` (~3.4%) es representativo. La categorización como `'Unknown'` permite mantener a estos individuos en el estudio, evitando un sesgo hacia alumnos de los que sí se dispone de información privada compleja.
*   **Saneamiento de la Variable Objetivo:** La eliminación de los registros de evaluación sin `score` es marginal (0.01%) y necesaria para asegurar que el modelo se entrene exclusivamente con etiquetas de rendimiento verificadas.
*   **Alineación Temporal Coherente:** Se confirma que la fecha de registro varía según la presentación (`code_presentation`), lo que valida que la imputación debe ser siempre **condicional al curso** y nunca global.

Estas decisiones garantizan un dataset final robusto, donde el **99.9% de los estudiantes originales** y sus interacciones son aprovechados para el entrenamiento de los modelos predictivos.


## 1. Distribución de la Variable Objetivo (final_result)
Es crucial saber si tenemos clases desbalanceadas, ya que esto afectará a las métricas del modelo futuro.

In [ ]:
# Traducción de etiquetas para la gráfica
translation_map = {
    'Pass': 'Aprobado',
    'Distinction': 'Sobresaliente',
    'Fail': 'Suspenso',
    'Withdrawn': 'Abandono'
}

df_students['Resultado'] = df_students['final_result'].map(translation_map)

plt.figure(figsize=(9, 6))
ax = sns.countplot(
    data=df_students, 
    x='Resultado', 
    palette='viridis', 
    order=['Aprobado', 'Sobresaliente', 'Suspenso', 'Abandono']
)

# Añadir etiquetas de valor encima de las barras
for i in ax.containers:
    ax.bar_label(i)

plt.title("Distribución de Estudiantes por Resultado Final", fontsize=14)
plt.xlabel("Resultado Académico", fontsize=12)
plt.ylabel("Cantidad de Estudiantes", fontsize=12)
plt.show()

**Observaciones:** El análisis de la distribución de clases revela un escenario complejo que justifica la necesidad de modelos avanzados. Si planteamos el problema de forma binaria como "Persistencia" (Aprobado + Sobresaliente ≈ 15.3k) frente a "Riesgo" (Suspenso + Abandono ≈ 17.2k), el dataset se encuentra razonablemente balanceado, lo cual es ideal para el entrenamiento. Sin embargo, al desglosar el riesgo, observamos que el Abandono (10.1k) es significativamente más frecuente que el Suspenso académico (7k). Esta disparidad refuerza la hipótesis de que la deserción es el problema principal a atajar y sugiere que un modelo multiclasificador será más efectivo para distinguir entre quien se va prematuramente y quien falla por rendimiento.

## 2. Estudio de interacciones de los estudiantes

El dataset OULAD posee una naturaleza intrínsecamente secuencial que lo distingue de los conjuntos de datos estáticos tradicionales. Tanto las interacciones diarias en el VLE (date, sum_click) como las evaluaciones (date_submitted, score) se distribuyen a lo largo del tiempo, conformando la "huella digital" del aprendizaje de cada estudiante semana a semana. Esta estructura temporal es el fundamento ideal para la aplicación de arquitecturas basadas en Transformers. Al igual que estos modelos procesan palabras en una oración para entender el contexto, aquí procesaremos la secuencia de comportamientos semanales del alumno para predecir su trayectoria futura, permitiendo al sistema capturar dependencias a largo plazo (e.g., una baja actividad en la semana 2 que repercute en el examen de la semana 10) que modelos clásicos pasarían por alto.

### 2.1 Clics promedios diarios

Si miramos a un solo estudiante, su gráfica podría ser engañosa: quizás ese alumno solo estudia los domingos, o quizás se puso enfermo una semana. Al usar el promedio de todos, lo que emerge es el ritmo real del curso. Verás que todos los alumnos (en promedio) suben su actividad justo antes de una entrega. El promedio nos permite ver los hitos del calendario académico que afectan a la masa.

In [ ]:
# 1. Calculamos el promedio de clics diarios de todos los estudiantes
daily_activity = df_interactions.groupby('date')['sum_click'].mean().reset_index()

# 2. Modificamos el filtro para incluir la actividad PREVIA (desde el día -25)
# Mantenemos el límite de 240 para no desvirtuar el final
daily_activity_filtered = daily_activity[(daily_activity['date'] >= -25) & (daily_activity['date'] <= 240)]

plt.figure(figsize=(15, 5))

# 3. Dibujamos la línea de actividad
plt.plot(daily_activity_filtered['date'], daily_activity_filtered['sum_click'], color='#e74c3c', linewidth=1)

# 4. Añadimos una línea vertical en el Día 0 para marcar el inicio oficial
plt.axvline(x=0, color='black', linestyle='--', alpha=0.8, label='Inicio oficial del curso (Día 0)')

plt.title('Promedio de Interacciones Diarias (Incluyendo Periodo Pre-Lectivo)', fontsize=14)
plt.xlabel('Día del curso (Días negativos = actividad antes del inicio)', fontsize=12)
plt.ylabel('Promedio de Clics', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

**Observaciones:**  Se aprecia un alto nivel de ruido y volatilidad en la actividad diaria (efecto sierra), lo que dificultaría el entrenamiento de un modelo de IA. También se confirma actividad relevante en los días previos al inicio oficial (Día < 0). Esta inestabilidad justifica la agregación semanal, necesaria para limpiar el ruido y capturar patrones de comportamiento más robustos y útiles para el modelo.

### 2.2 clics promedio semanales

In [ ]:
# 1. Preparamos los datos
# Actividad Diaria
daily_avg = df_interactions.groupby('date')['sum_click'].mean().reset_index()

# Actividad Semanal (Agrupamos primero por estudiante/semana y luego promediamos)
df_interactions['week'] = df_interactions['date'] // 7
weekly_avg = df_interactions.groupby(['id_student', 'week'])['sum_click'].sum().reset_index()
weekly_trend = weekly_avg.groupby('week')['sum_click'].mean().reset_index()

# 2. Configuración de la comparativa
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=False)

# Gráfica Superior: Ruido Diario
ax1.plot(daily_avg['date'], daily_avg['sum_click'], color='#e74c3c', linewidth=1, alpha=0.8)
ax1.axvline(x=0, color='black', linestyle='--')
ax1.set_title('ANÁLISIS DIARIO: Ruido de alta frecuencia (difícil de modelar)', fontsize=14)
ax1.set_ylabel('Promedio Clics/Día')
ax1.grid(True, alpha=0.3)

# Gráfica Inferior: Señal Semanal
# Multiplicamos X por 7 para mantener la misma escala temporal que los días
ax2.plot(weekly_trend['week'] * 7, weekly_trend['sum_click'], color='#3498db', linewidth=3, marker='o')
ax2.axvline(x=0, color='black', linestyle='--')
ax2.set_title('ANÁLISIS SEMANAL: Señal limpia y patrones identificables', fontsize=14)
ax2.set_ylabel('Promedio Clics/Semana')
ax2.set_xlabel('Días del curso')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Observaciones:** El contraste visual es definitivo: mientras que la serie diaria está saturada de variaciones bruscas (ruido) que no aportan valor predictivo, la agregación semanal filtra esas fluctuaciones y revela la estructura real del curso.

Al usar semanas, transformamos datos caóticos en una serie temporal limpia donde destacan los hitos académicos. Esta es la base que permitirá al modelo (Transformer/RNN) aprender de forma eficiente, evitando que se pierda en detalles irrelevantes del día a día y centrando su capacidad en la evolución del compromiso del estudiante.

In [ ]:
## --- ANÁLISIS POR RENDIMIENTO: ¿Quién deja de estudiar en las zonas críticas? ---

# 1. Enriquecer Interacciones con el Resultado Final
student_weekly = df_interactions.groupby(['id_student', 'week'])['sum_click'].sum().reset_index()
student_weekly = student_weekly.merge(df_students[['id_student', 'final_result']], on='id_student', how='inner')

# Traducimos etiquetas
traduccion = {'Pass': 'Aprobado', 'Distinction': 'Sobresaliente', 'Fail': 'Suspenso', 'Withdrawn': 'Abandono'}
student_weekly['Resultado'] = student_weekly['final_result'].map(traduccion)

# 2. Calcular Promedios por Grupo
trends_by_result = student_weekly.groupby(['Resultado', 'week'])['sum_click'].mean().reset_index()

# <<< RECALCULO DE SEMANAS CRÍTICAS (Para autonomía de celda) >>>
df_assessments['week_deadline'] = df_assessments['date'] // 7
deadline_counts = df_assessments['week_deadline'].value_counts()
semanas_criticas_local = deadline_counts[deadline_counts > deadline_counts.mean()].index

# 3. Visualización
plt.figure(figsize=(15, 7))

# Líneas de Tendencia
custom_colors = {'Abandono': '#e74c3c', 'Suspenso': '#f39c12', 'Aprobado': '#3498db', 'Sobresaliente': '#2ecc71'}
sns.lineplot(data=trends_by_result, x='week', y='sum_click', hue='Resultado', palette=custom_colors, linewidth=2.5, marker='o')

# Zonas de Alta Carga
for semana in semanas_criticas_local: # Usamos la variable local
    plt.axvspan(semana - 0.5, semana + 0.5, color='gray', alpha=0.15)

# Estilos
plt.title('Huella Digital del Éxito: Actividad Semanal según Resultado Final', fontsize=16)
plt.ylabel('Promedio de Clics por Estudiante')
plt.xlabel('Semana del Curso')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(title='Resultado Final', loc='upper right')
plt.xlim(-2, 39)

plt.tight_layout()
plt.show()

**Observaciones:** Se aprecia una distorsión de los datos en las últimas semanas del grupo de Abandono, con un pico de actividad atípico. Esto se debe a la baja representatividad estadística (pocos alumnos residuales), donde acciones individuales —como entrar a descargar materiales antes de irse o consultar notas finales en el "cierre de actas"— disparan el promedio. Dada esta volatilidad por falta de volumen, es recomendable truncar o normalizar los datos finales antes de alimentar el modelo de IA para evitar sesgos.

In [ ]:
# --- ANÁLISIS CUALITATIVO: ¿En qué invierten su tiempo? (Clics por Tipo de Recurso) ---

# 1. Preparar datos
# Hacemos merge para tener interactividad + resultado
merged = df_interactions.merge(df_students[['id_student', 'final_result']], on='id_student', how='inner')

# Traducimos resultados
traduccion = {'Pass': 'Aprobado', 'Distinction': 'Sobresaliente', 'Fail': 'Suspenso', 'Withdrawn': 'Abandono'}
merged['Resultado'] = merged['final_result'].map(traduccion)

# 2. Calcular Distribución de Clics
# Agrupamos por Resultado y Tipo de Actividad
activity_dist = merged.groupby(['Resultado', 'activity_type'])['sum_click'].sum().reset_index()

# Calculamos el total de clics por grupo para sacar porcentajes
total_clicks_by_result = merged.groupby('Resultado')['sum_click'].sum().reset_index()
activity_dist = activity_dist.merge(total_clicks_by_result, on='Resultado', suffixes=('', '_total'))
activity_dist['percentage'] = (activity_dist['sum_click'] / activity_dist['sum_click_total']) * 100

# 3. Pivotar para Heatmap
heatmap_data = activity_dist.pivot(index='activity_type', columns='Resultado', values='percentage')
# Ordenamos las actividades por popularidad global para que el gráfico sea legible
top_activities = merged.groupby('activity_type')['sum_click'].sum().sort_values(ascending=False).index
heatmap_data = heatmap_data.reindex(top_activities)
# Reordenamos columnas lógicamente
heatmap_data = heatmap_data[['Abandono', 'Suspenso', 'Aprobado', 'Sobresaliente']]

# 4. Visualización
plt.figure(figsize=(10, 12))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=.5, cbar_kws={'label': '% del Total de Clics del Grupo'})
plt.title('Perfil de Navegación: ¿Qué consumen los diferentes estudiantes?', fontsize=14)
plt.ylabel('Tipo de Recurso')
plt.xlabel('Resultado Final')
plt.show()


## Hallazgos del Análisis Temporal

*   **Brecha de Actividad**: Se observa una clara separación entre estudiantes excelentes y en riesgo durante el ecuador del curso (Semanas 15-25). Los alumnos de alto rendimiento son proactivos, mientras que los de bajo rendimiento son reactivos a las entregas.
*   **Limitación de Hitos Globales**: Los picos de actividad no siempre coinciden con los *deadlines* globales de la universidad, lo que sugiere que para la IA será vital capturar patrones específicos de cada módulo.
*   **Señal vs Ruido**: La agregación semanal es fundamental para eliminar la volatilidad diaria y permitir que el modelo capture la trayectoria real del estudiante.

In [ ]:
# --- ANÁLSIS DE PATRONES DE ABANDONO VS RENDIMIENTO ---
# Objetivo: Comprender la dinámica temporal y el comportamiento diferencial entre
# estudiantes que tienen éxito y aquellos que abandonan. Esta información será
# clave para diseñar el sistema de alerta temprana basado en Transformers.
# --- TRIPLE DESGLOSE: Top 3 Cursos por Resultado Final ---
# 1. Identificar Top 3 Cursos Masivos
top_3_courses = df_students.groupby(['code_module', 'code_presentation']).size().sort_values(ascending=False).head(3)
# 2. Configuración Visual
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
result_colors = {'Aprobado': '#3498db', 'Sobresaliente': '#2ecc71', 'Suspenso': '#f39c12', 'Abandono': '#e74c3c'}
traduccion = {'Pass': 'Aprobado', 'Distinction': 'Sobresaliente', 'Fail': 'Suspenso', 'Withdrawn': 'Abandono'}
# 3. Bucle para generar los 3 gráficos
for i, ((curso, semestre), _) in enumerate(top_3_courses.items()):
    ax = axes[i]
    
    # A. Filtrar Datos Específicos
    # Estudiantes de este curso concreto con su nota final
    sub_students = df_students[(df_students['code_module'] == curso) & (df_students['code_presentation'] == semestre)][['id_student', 'final_result']]
    # Interacciones de este curso
    sub_inter = df_interactions[(df_interactions['code_module'] == curso) & (df_interactions['code_presentation'] == semestre)]
    # Deadlines de este curso
    sub_assess = df_assessments[(df_assessments['code_module'] == curso) & (df_assessments['code_presentation'] == semestre)]
    
    # B. Merge para tener 'final_result' en las interacciones
    merged_data = sub_inter.merge(sub_students, on='id_student', how='inner')
    merged_data['Resultado'] = merged_data['final_result'].map(traduccion)
    merged_data['week'] = merged_data['date'] // 7
    
    # C. Agrupar (Media de clics por resultado y semana)
    # Primero sumamos por alumno/semana, luego promediamos por grupo
    student_level = merged_data.groupby(['id_student', 'Resultado', 'week'])['sum_click'].sum().reset_index()
    trend_data = student_level.groupby(['Resultado', 'week'])['sum_click'].mean().reset_index()
    
    # D. Pintar Líneas por Resultado
    sns.lineplot(data=trend_data, x='week', y='sum_click', hue='Resultado', palette=result_colors, ax=ax, linewidth=2, marker='o')
    
    # E. Pintar Deadlines (Fondo Rojo)
    deadlines = (sub_assess['date'] // 7).unique()
    for d in deadlines:
        ax.axvline(x=d, color='gray', linestyle=':', alpha=0.5)
        ax.axvspan(d - 0.5, d + 0.5, color='gray', alpha=0.1)
        
    # F. Cosmética
    ax.set_title(f'Dinámicas en {curso} ({semestre})', fontsize=12, fontweight='bold')
    ax.set_ylabel('Clics/Alumno')
    ax.legend(loc='upper right', fontsize='small', ncol=2)
    ax.grid(True, alpha=0.3)
plt.xlabel('Semana del Curso')
plt.xlim(-2, 39)
plt.suptitle('Análisis de Comportamiento por Éxito en los Cursos Principales', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Usamos la columna 'date' (fecha deadline oficial) si existe, o 'date_submitted'
if 'date' in train_assessments.columns:
    target_date = 'date'
else:
    target_date = 'date_submitted' # Fallback si no está la oficial

# Gráfico de violín para ver la distribución temporal
plt.figure(figsize=(10, 6))
sns.violinplot(x='assessment_type', y=target_date, data=train_assessments, palette='viridis')
plt.title("Distribución Temporal de Tipos de Evaluación")
plt.ylabel("Día del Curso (Deadline)")
plt.xlabel("Tipo de Evaluación")
plt.grid(axis='y', alpha=0.3)

# Dibujar líneas de referencia (Semana 0 y Semana 35)
plt.axhline(0, color='r', linestyle='--', alpha=0.5, label='Inicio Curso')
plt.axhline(240, color='k', linestyle='--', alpha=0.5, label='Fin Curso (~Day 240)')
plt.legend()
plt.show()

# Estadísticas descriptivas
print("Resumen de Fechas (Días) por Tipo:")
print(train_assessments.groupby('assessment_type')[target_date].describe()[['min', 'mean', 'max', 'count']])

## 3. Correlación: Clics Totales vs Nota Media
¿Estudiar más (más clics) garantiza mejor nota?

In [ ]:
# Calculamos clics totales por estudiante
total_clicks = df_interactions.groupby('id_student')['sum_click'].sum().reset_index()
total_clicks.rename(columns={'sum_click': 'total_clicks'}, inplace=True)

# Calculamos nota media por estudiante (en evaluaciones puntuables)
avg_score = df_assessments.groupby('id_student')['score'].mean().reset_index()
avg_score.rename(columns={'score': 'mean_score'}, inplace=True)

# Unimos
analysis_df = pd.merge(total_clicks, avg_score, on='id_student')
analysis_df = pd.merge(analysis_df, df_students[['id_student', 'final_result']], on='id_student')

# Traducción para visualización
analysis_df['Resultado'] = analysis_df['final_result'].map(translation_map)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=analysis_df, x='total_clicks', y='mean_score', hue='Resultado', alpha=0.6, palette='viridis')
plt.title("Relación entre Clics Totales y Nota Media")
plt.xscale('log') # Escala logarítmica porque suele haber outliers con muchísimos clics
plt.xlabel("Clics Totales (Escala Logarítmica)")
plt.ylabel("Nota Media")
plt.show()

In [ ]:
# --- ANÁLISIS DEMOGRÁFICO: Estilo Barras Agrupadas por Quintil ---

# 1. Preparar Datos y Agrupar en 5 Quintiles
demo_df = df_students.dropna(subset=['imd_band']).copy()

im_mapping = {
    '0-10%': '0-20% (Muy Baja)', '10-20': '0-20% (Muy Baja)',
    '20-30%': '20-40% (Baja)',   '30-40%': '20-40% (Baja)',
    '40-50%': '40-60% (Media)',  '50-60%': '40-60% (Media)',
    '60-70%': '60-80% (Alta)',   '70-80%': '60-80% (Alta)',
    '80-90%': '80-100% (Muy Alta)', '90-100%': '80-100% (Muy Alta)'
}
demo_df['Quintil_Riqueza'] = demo_df['imd_band'].map(im_mapping)
orden_quintiles = ['0-20% (Muy Baja)', '20-40% (Baja)', '40-60% (Media)', '60-80% (Alta)', '80-100% (Muy Alta)']

# 2. Calcular Porcentajes (Formato Largo para Seaborn)
# Contamos alumnos por Quintil y Resultado
grouped = demo_df.groupby(['Quintil_Riqueza', 'final_result']).size().reset_index(name='count')

# Calculamos el total por Quintil para sacar el %
total_by_quintil = grouped.groupby('Quintil_Riqueza')['count'].transform('sum')
grouped['percentage'] = (grouped['count'] / total_by_quintil) * 100

# Traducimos para la leyenda
traduccion = {'Pass': 'Aprobado', 'Distinction': 'Sobresaliente', 'Fail': 'Suspenso', 'Withdrawn': 'Abandono'}
grouped['Resultado'] = grouped['final_result'].map(traduccion)

# 3. Visualización (Barras Agrupadas)
plt.figure(figsize=(14, 7))

colores_custom = {'Abandono': '#e74c3c', 'Suspenso': '#f39c12', 'Aprobado': '#3498db', 'Sobresaliente': '#2ecc71'}

sns.barplot(
    data=grouped,
    x='Quintil_Riqueza',
    y='percentage',
    hue='Resultado',
    hue_order=['Abandono', 'Suspenso', 'Aprobado', 'Sobresaliente'],
    palette=colores_custom,
    order=orden_quintiles # Aseguramos el orden lógico de pobreza a riqueza
)

# Cosmética
plt.title('Impacto Socioeconómico: Distribución de Resultados por Nivel de Riqueza', fontsize=16)
plt.xlabel('Nivel de Riqueza de la Región (Quintiles)')
plt.ylabel('% de Estudiantes del Grupo')
plt.legend(title='Resultado Final', bbox_to_anchor=(1.01, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- TRIPLETE FINAL (Versión Mejorada con Desglose Semanal por Éxito) ---

# CONFIGURACIÓN GENERAL
fig = plt.figure(figsize=(20, 18))
gs = fig.add_gridspec(3, 2) 

# ---------------------------------------------------------
# GRÁFICA 1: SUPERVIVENCIA
# ---------------------------------------------------------
ax1 = fig.add_subplot(gs[0, :])
withdrawals = df_students[df_students['date_unregistration'].notna()]['date_unregistration']
sns.histplot(withdrawals, kde=True, element="step", fill=False, cumulative=True, color='red', ax=ax1, lw=3)
ax1.clear()
sns.histplot(withdrawals, bins=50, color='#e74c3c', kde=True, ax=ax1)
ax1.set_title('Cronología del Abandono: ¿Cuándo se rinden los estudiantes?', fontsize=16)
ax1.set_ylabel('Número de Abandonos')
ax1.axvline(0, color='black', linestyle='--')

# ---------------------------------------------------------
# GRÁFICA 2: CORRELACIÓN PRIMERA NOTA
# ---------------------------------------------------------
ax2 = fig.add_subplot(gs[1, 0]) 
first_scores = df_assessments.sort_values('date_submitted').groupby('id_student').first().reset_index()
score_corr = first_scores.merge(df_students[['id_student', 'final_result']], on='id_student')
traduccion_nota = {'Pass': 'Aprobado', 'Distinction': 'Sobresaliente', 'Fail': 'Suspenso', 'Withdrawn': 'Abandono'}
score_corr['Resultado'] = score_corr['final_result'].map(traduccion_nota)
sns.boxplot(x='Resultado', y='score', data=score_corr, order=['Abandono', 'Suspenso', 'Aprobado', 'Sobresaliente'], palette='viridis', ax=ax2)
ax2.set_title('Impacto de la Primera Nota', fontsize=14)

# ---------------------------------------------------------
# GRÁFICA 3: HÁBITOS SEMANALES DESGLOSADOS
# ---------------------------------------------------------
ax3 = fig.add_subplot(gs[1, 1])

# 1. Preparamos datos enriquecidos (Interacción + Resultado)
# Usamos una muestra más grande para que el desglose sea fiable
sample_int_enriched = df_interactions.sample(1500000, random_state=42).merge(
    df_students[['id_student', 'final_result']], on='id_student', how='inner'
)
sample_int_enriched['dia_semana'] = sample_int_enriched['date'] % 7
sample_int_enriched['Resultado'] = sample_int_enriched['final_result'].map(traduccion_nota)

# 2. Agrupamos: Clics totales por (Día, Resultado)
weekly_breakdown = sample_int_enriched.groupby(['dia_semana', 'Resultado'])['sum_click'].sum().reset_index()

# 3. Normalizamos (Opcional, pero recomendado):
# Como hay muchos más aprobados que suspensos, las barras azules serán gigantes siempre.
# Para ver PATRONES horariros, calculamos el % de actividad diaria DENTRO de cada grupo.
# (Ej: Del 100% de clics de los Sobresalientes, ¿cuánto cae en Domingo?)
total_clicks_per_group = weekly_breakdown.groupby('Resultado')['sum_click'].transform('sum')
weekly_breakdown['pct_click'] = (weekly_breakdown['sum_click'] / total_clicks_per_group) * 100

# 4. Visualización con Barras Agrupadas (hue)
# Orden de días para que quede bonito
colores_custom = {'Abandono': '#e74c3c', 'Suspenso': '#f39c12', 'Aprobado': '#3498db', 'Sobresaliente': '#2ecc71'}

sns.barplot(
    data=weekly_breakdown, 
    x='dia_semana', 
    y='pct_click', # Usamos porcentaje para comparar hábitos, no volumen bruto
    hue='Resultado', 
    hue_order=['Abandono', 'Suspenso', 'Aprobado', 'Sobresaliente'],
    palette=colores_custom,
    ax=ax3
)

ax3.set_title('Ritmo Semanal Normalizado: ¿Quién estudia los fines de semana?', fontsize=14)
ax3.set_xlabel('Día del Ciclo (0-6)')
ax3.set_ylabel('% de la Actividad Semanal del Grupo')
ax3.set_xticklabels(['Día 1', 'Día 2', 'Día 3', 'Día 4', 'Día 5', 'Día 6', 'Día 7'])
ax3.legend(title='Resultado', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

**Observaciones:** Es un problema serio que no podamos detectar el 40% de estudiantes que abandonan en el dia cero, necesitamos descubrir patrones antes del dia cero que nos permitan averiguar cosillas.

Conclusión: "Estudiar en fin de semana" NO es predictor de éxito. (Rompe el mito del estudiante sacrificado).Decisión de Ingeniería: Esto valida al 100% nuestra decisión de hacer Agregación Semanal.
Si hubiéramos visto patrones distintos, estaríamos obligados a alimentar a la IA con datos diarios (lunes, martes...).
Como el patrón es idéntico, podemos sumar todo a la semana (clics_semana_X) y ahorramos un 85% de complejidad computacional sin perder ni una pizca de capacidad predictiva.



In [ ]:
# --- OPERACIÓN RESCATE DÍA 0: Detectando patrones Pre-Arranque ---

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# ---------------------------------------------------------
# 1. EL FACTOR ANTELACIÓN (Date Registration)
# ---------------------------------------------------------
# date_registration suele ser negativo (días antes del curso).
# Vamos a ver si los que se registran MUY pronto o MUY tarde abandonan más.
sns.kdeplot(data=df_students[df_students['final_result'] == 'Withdrawn']['date_registration'], 
            label='Abandonos', color='red', fill=True, ax=axes[0])
sns.kdeplot(data=df_students[df_students['final_result'] == 'Distinction']['date_registration'], 
            label='Sobresalientes', color='green', fill=True, ax=axes[0])
axes[0].set_title('Antelación de Matrícula: ¿Influye cuándo te apuntas?', fontsize=12)
axes[0].set_xlabel('Días antes del inicio (0 = Inicio)')
axes[0].legend()
axes[0].axvline(0, color='black', linestyle='--')

# ---------------------------------------------------------
# 2. EL FACTOR "MOCHILA" (Intentos Previos)
# ---------------------------------------------------------
# ¿Los reincidentes abandonan más al principio?
prev_attempts = df_students.groupby(['num_of_prev_attempts', 'final_result']).size().unstack(fill_value=0)
prev_attempts_pct = prev_attempts.div(prev_attempts.sum(axis=1), axis=0) * 100
# Solo mostramos la tasa de abandono
prev_attempts_pct['Withdrawn'].plot(kind='bar', color='#e74c3c', ax=axes[1])
axes[1].set_title('Reincidencia: Tasa de Abandono según Intentos Previos', fontsize=12)
axes[1].set_ylabel('% de Abandonos')
axes[1].set_xlabel('Número de Intentos Previos')

# ---------------------------------------------------------
# 3. EL MIEDO ESCÉNICO (Actividad Pre-Curso)
# ---------------------------------------------------------
# Calculamos si tuvieron actividad ANTES del día 0
pre_clicks = df_interactions[df_interactions['date'] < 0].groupby('id_student')['sum_click'].sum().reset_index()
pre_clicks.rename(columns={'sum_click': 'pre_clicks'}, inplace=True)

# Unimos con estudiantes (left join para tener a los que tienen 0 clics)
std_pre = df_students.merge(pre_clicks, on='id_student', how='left').fillna(0)

# Clasificamos: ¿Hizo Clics Pre-Curso? SI / NO
std_pre['Investigated_Platform'] = std_pre['pre_clicks'] > 0
# Vemos tasa de abandono según si investigaron o no
curiosity_gap = std_pre.groupby(['Investigated_Platform', 'final_result']).size().unstack()
curiosity_gap_pct = curiosity_gap.div(curiosity_gap.sum(axis=1), axis=0) * 100

curiosity_gap_pct[['Withdrawn', 'Fail', 'Pass', 'Distinction']].plot(kind='bar', stacked=True, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'], ax=axes[2])
axes[2].set_title('Curiosidad: ¿Entraron antes de empezar?', fontsize=12)
axes[2].set_xticklabels(['No (Entraron el día 0 o nunca)', 'Sí (Investigaron antes)'], rotation=0)

plt.tight_layout()
plt.show()


### **3. Análisis de Sesgos y Equidad Algorítmica (Fairness)**

En el desarrollo de sistemas de aprendizaje automático aplicados a la educación, la **Equidad (Fairness)** es un requisito ético fundamental. Un modelo de IA puede perpetuar o incluso amplificar las desigualdades históricas si no se analizan y mitigan los sesgos presentes en los datos demográficos.

En esta sección, se evalúa la distribución de la variable objetivo (`final_result`) en relación con los **atributos protegidos** y factores demográficos clave, con el fin de identificar posibles disparidades en los resultados educativos.

#### **Dimensiones de Análisis de Sesgo:**
1.  **Género (`gender`):** Evaluación de disparidades en el rendimiento académico entre hombres y mujeres.
2.  **Discapacidad (`disability`):** Análisis de la brecha de éxito para estudiantes que requieren adaptaciones.
3.  **Capital Cultural (`highest_education`):** Investigación del impacto del nivel educativo previo en la probabilidad de éxito actual.
4.  **Entorno Socioeconómico (`imd_band`):** Relación entre la privación económica de la región de origen y los resultados académicos.

In [ ]:
import matplotlib.pyplot as plt

# Definimos los 3 atributos principales para comparar en serie
protected_attributes = ['gender', 'disability', 'highest_education']

# Creamos 1 fila con 3 columnas
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

for i, attr in enumerate(protected_attributes):
    # Cálculo de porcentajes
    props = (df_students.groupby(attr, observed=False)['final_result']
             .value_counts(normalize=True)
             .unstack() * 100)
    
    # Orden lógico para educación
    if attr == 'highest_education':
        order = ['No Formal quals', 'Lower Than A Level', 'A Level or Equivalent', 'HE Qualification', 'Post Graduate Qualification']
        props = props.reindex([o for o in order if o in props.index])

    # Plot (quitamos la leyenda individual con legend=False)
    props.plot(kind='bar', stacked=True, ax=axes[i], colormap='viridis', alpha=0.8, legend=False)
    
    # Formateo de cada subplot
    axes[i].set_title(f"Por {attr.replace('_', ' ').capitalize()}", fontsize=13, fontweight='bold')
    axes[i].set_ylabel("Porcentaje (%)" if i == 0 else "")
    axes[i].set_xlabel("")
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(axis='y', linestyle='--', alpha=0.3)

# --- RECICLAJE DE LEYENDA ---
# Tomamos los nombres de la leyenda del primer gráfico
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, title="Resultado Final", loc='center right', bbox_to_anchor=(1.1, 0.5))

plt.suptitle("Análisis de Equidad: Brechas en Resultados Educativos", fontsize=16, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

#### **Interpretación Crítica de los Sesgos Detectados**

El análisis visual revela disparidades significativas que el modelo de aprendizaje profundo podría explotar de manera automática:

*   **Sesgo de Capital Cultural:** Existe una correlación lineal entre el nivel educativo previo (`highest_education`) y la tasa de éxito (`Distinction`). Una red neuronal identificará este patrón como un atajo predictivo fuerte. El riesgo académico es que el modelo penalice a perfiles con menor formación previa, ignorando su progreso real durante el curso.
*   **Brecha de Accesibilidad:** Los estudiantes con discapacidad presentan una franja de abandono (`Withdrawn`) ligeramente superior. Si el modelo utiliza esta variable sin procesar, podría generar alertas sesgadas que no consideren las barreras de accesibilidad externas al estudiante.
*   **Equidad de Género:** En esta muestra, las distribuciones son notablemente similares, lo que sugiere que el atributo `gender` podría tener un peso neutral en la predicción, reduciendo el riesgo de sesgo por sexo.

**Implicación para el Modelo:** Estos hallazgos justifican la necesidad de evaluar el modelo no solo mediante métricas globales (Accuracy/F1), sino también mediante **métricas de equidad (Fairness Metrics)**, asegurando que la precisión sea consistente en todos los grupos sociodemográficos.